In [2]:
import numpy as np 
from tqdm.notebook import tqdm 
import os 
import pandas as pd
import pickle
import jams
import torch

print(torch.cuda.is_available())
print(torch.cuda.current_device())
os.chdir('/home/lukerowe/projects/def-gtzan/lukerowe/ismir2021_supp/notebooks')

True
0


In [3]:
# First we get the track weights for evaluation
# Note that since WCSR metrics are evaluated at the frame-level then
# we must weight each track by the len(track) / len(all_tracks)
for i in range(5):
    fold_ids = pd.read_csv(os.path.join('../data/humphrey_bello_folds', "test{:02d}.csv".format(i)), index_col=0, header=None)
    track_weights = {}
    tot_len = 0
    for j in tqdm(fold_ids.index):
        j = int(j)
        jam = jams.load(os.path.join('../../MARL/marl_jams', "{}.jams".format(j)))
        track_len = float(jam["file_metadata"]['duration'])
        track_weights[j] = track_len
        tot_len += track_len 
    
    for j in fold_ids.index:
        j = int(j)
        track_weights[j] /= tot_len

    with open("track_weights_fold{:02d}.pkl".format(i), "wb") as f:
        pickle.dump(track_weights, f)

In [25]:
# We map track ids from original humphrey-bello folds to 0,1,...,1216
with open("../data/filename_mapping.pkl", "rb") as f:
    filename_mapping = pickle.load(f)

print(filename_mapping)

{'TRDLAJY149E2BBA3F4': 0, 'TRVGNVA149E3CCE4FC': 1, 'TRKCXCG149E3DF10FF': 2, 'TRWGBHX149E35D0876': 3, 'TRFSZWH149E330911A': 4, 'TRLUVHO149E3B65B46': 5, 'TRCRNBV149E2E04318': 6, 'TRJYRLE149E34E8993': 7, 'TRPBIIJ127FADB1A1F': 8, 'TRLNINW149E2E6FB20': 9, 'TRKPDVF149E3B98AF2': 10, 'TRUEMQB149E3465DA5': 11, 'TREXPPD149E344AB31': 12, 'TRUVBWF149E3D3E15D': 13, 'TRTVJNY149E2E23D7B': 14, 'TRHDNSP149E39666AE': 15, 'TRVPCAF149E3ACD57A': 16, 'TRVAGZA127F92CD4AC': 17, 'TRYRCHF149E351E1F0': 18, 'TRTSRWL127F92CD8E9': 19, 'TRIMYHG149E3A00712': 20, 'TRLMIXE149E3898756': 21, 'TRHSELP149E3A2933D': 22, 'TRKEOPJ149E356916C': 23, 'TRGANRB127F8603C9B': 24, 'TRBFASH149E341F334': 25, 'TRWLKQC149E374AE38': 26, 'TRRPGRC149E37CE5AF': 27, 'TRGRMUK149E329D564': 28, 'TRCYFSX149E38E8293': 29, 'TRCMNRR149E3BDE1E7': 30, 'TRMTSHF149E3179C00': 31, 'TRMOOXL149E3A2FFCE': 32, 'TROQOWJ127F9D570A3': 33, 'TRHPTPU149E3CF2AEA': 34, 'TRCWTCA149E38A650A': 35, 'TRFQQPQ149E3C5A752': 36, 'TRQTTVG149E39099B9': 37, 'TRDVEWY149E3DE16C2':

In [33]:
# First, we evaluate on traditional WCSR metrics
def load_results(model_name, is_mcfee, vocab_size_is_301):
    root_results = []
    thirds_results = []
    triads_results = []
    sevenths_results = []
    tetrads_results = []
    majmin_results = []
    mirex_results = []

    for i in range(5):
        with open("track_weights_fold{:02d}.pkl".format(i), "rb") as f:
            track_weights = pickle.load(f)

        
        if vocab_size_is_301:
            filename = os.path.join('../results/{}/fold{:02d}_test_301.csv'.format(model_name, i))
        else:
            filename = os.path.join('../results/{}/fold{:02d}_test.csv'.format(model_name, i))
        results = pd.read_csv(filename, index_col=0)

        index = list(results.index)
        for i, ind in enumerate(index):
            if is_mcfee:
                # map humphrey-bello track id to corresponding id {0,1,...,1216}
                ind = filename_mapping[ind]
            results.iloc[i,:] = results.iloc[i,:] * track_weights[ind]

        results = results[['root', 'thirds', 'triads', 'sevenths', 'tetrads', 'majmin', 'mirex']]
        
        weighted_results = results.sum(axis=0)

        root_results.append(weighted_results['root'])
        thirds_results.append(weighted_results['thirds'])
        triads_results.append(weighted_results['triads'])
        sevenths_results.append(weighted_results['sevenths'])
        tetrads_results.append(weighted_results['tetrads'])
        majmin_results.append(weighted_results['majmin'])
        mirex_results.append(weighted_results['mirex'])

    root_value = float(np.mean(np.array(root_results)))
    thirds_value = float(np.mean(np.array(thirds_results)))
    triads_value = float(np.mean(np.array(triads_results)))
    sevenths_value = float(np.mean(np.array(sevenths_results)))
    tetrads_value = float(np.mean(np.array(tetrads_results)))
    majmin_value = float(np.mean(np.array(majmin_results)))
    mirex_value = float(np.mean(np.array(mirex_results)))

    print(model_name)
    print("\nRoot Thirds Triads Sevenths Tetrads MajMin MIREX")
    print("{0:.3f} {1:.3f} {2:.3f} {3:.3f} {4:.3f} {5:.3f} {6:.3f}".format(root_value, thirds_value, triads_value, sevenths_value, tetrads_value, majmin_value, mirex_value))


In [38]:
load_results('CT', False, False)

CT

Root Thirds Triads Sevenths Tetrads MajMin MIREX
0.838 0.809 0.767 0.714 0.650 0.826 0.832


In [39]:
load_results('CT-RE', False, False)

CT-RE

Root Thirds Triads Sevenths Tetrads MajMin MIREX
0.836 0.807 0.764 0.711 0.646 0.823 0.828


In [40]:
load_results('CT-RE-S', False, False)

CT-RE-S

Root Thirds Triads Sevenths Tetrads MajMin MIREX
0.831 0.802 0.759 0.709 0.644 0.819 0.825


In [41]:
load_results('CT-RE-S-C', False, False)

CT-RE-S-C

Root Thirds Triads Sevenths Tetrads MajMin MIREX
0.829 0.798 0.754 0.700 0.638 0.813 0.820


In [42]:
# CRNN evaluated over vocabulary of size 301
load_results('CRNN', False, True)

CRNN

Root Thirds Triads Sevenths Tetrads MajMin MIREX
0.837 0.803 0.759 0.694 0.630 0.822 0.812


In [43]:
# CRNN evaluated over vocabulary of size 169 (no X chord, as X does not have a structured decomposition)
# Note that evaluating over size 301 performs better
load_results('CRNN', False, False)

CRNN

Root Thirds Triads Sevenths Tetrads MajMin MIREX
0.834 0.801 0.757 0.691 0.626 0.821 0.809


In [44]:
load_results('MCFEE_2017', True, False)

MCFEE_2017

Root Thirds Triads Sevenths Tetrads MajMin MIREX
0.821 0.784 0.742 0.677 0.615 0.802 0.803


In [50]:
# Now, we evaluate the curriculum learning results (i.e.) on stratified folds)
def load_cl_results(model_name):
    
    with open("../results/{}/results.pkl".format(model_name), "rb") as f:
            results = pickle.load(f)
            
    print(model_name, "\n")
    print("acc_frame: {0:.3f}".format(results['wcsr']))
    print("acc_class: {0:.3f}".format(results['acta']))
    
    for cq in ['maj', 'min', '7', 'min7', 'maj7', 'sus4', 'maj6', 'min6', 'sus2', 'dim','aug', 'hdim7', 'dim7', 'minmaj7']:
        print("acc_quality, {0}: {1:.3f}".format(cq, results[cq]))

In [51]:
load_cl_results('CT_stratified')

CT_stratified 

acc_frame: 0.677
acc_class: 0.347
acc_quality, maj: 0.801
acc_quality, min: 0.637
acc_quality, 7: 0.518
acc_quality, min7: 0.576
acc_quality, maj7: 0.570
acc_quality, sus4: 0.277
acc_quality, maj6: 0.102
acc_quality, min6: 0.134
acc_quality, sus2: 0.034
acc_quality, dim: 0.365
acc_quality, aug: 0.236
acc_quality, hdim7: 0.357
acc_quality, dim7: 0.031
acc_quality, minmaj7: 0.031


In [53]:
load_cl_results('CT_CLBaseline_stratified')

CT_CLBaseline_stratified 

acc_frame: 0.647
acc_class: 0.427
acc_quality, maj: 0.727
acc_quality, min: 0.618
acc_quality, 7: 0.512
acc_quality, min7: 0.581
acc_quality, maj7: 0.604
acc_quality, sus4: 0.411
acc_quality, maj6: 0.250
acc_quality, min6: 0.264
acc_quality, sus2: 0.235
acc_quality, dim: 0.386
acc_quality, aug: 0.460
acc_quality, hdim7: 0.450
acc_quality, dim7: 0.209
acc_quality, minmaj7: 0.151


In [54]:
load_cl_results('CT_CLConvex_stratified')

CT_CLConvex_stratified 

acc_frame: 0.657
acc_class: 0.449
acc_quality, maj: 0.735
acc_quality, min: 0.626
acc_quality, 7: 0.537
acc_quality, min7: 0.595
acc_quality, maj7: 0.634
acc_quality, sus4: 0.406
acc_quality, maj6: 0.275
acc_quality, min6: 0.288
acc_quality, sus2: 0.261
acc_quality, dim: 0.395
acc_quality, aug: 0.517
acc_quality, hdim7: 0.476
acc_quality, dim7: 0.181
acc_quality, minmaj7: 0.229


In [55]:
load_cl_results('CT_CLLinear_stratified')

CT_CLLinear_stratified 

acc_frame: 0.658
acc_class: 0.439
acc_quality, maj: 0.741
acc_quality, min: 0.615
acc_quality, 7: 0.538
acc_quality, min7: 0.591
acc_quality, maj7: 0.638
acc_quality, sus4: 0.390
acc_quality, maj6: 0.283
acc_quality, min6: 0.264
acc_quality, sus2: 0.241
acc_quality, dim: 0.383
acc_quality, aug: 0.440
acc_quality, hdim7: 0.436
acc_quality, dim7: 0.240
acc_quality, minmaj7: 0.229


In [56]:
load_cl_results('CT_EC_stratified')

CT_EC_stratified 

acc_frame: 0.650
acc_class: 0.379
acc_quality, maj: 0.749
acc_quality, min: 0.601
acc_quality, 7: 0.536
acc_quality, min7: 0.587
acc_quality, maj7: 0.609
acc_quality, sus4: 0.322
acc_quality, maj6: 0.196
acc_quality, min6: 0.191
acc_quality, sus2: 0.188
acc_quality, dim: 0.299
acc_quality, aug: 0.273
acc_quality, hdim7: 0.371
acc_quality, dim7: 0.151
acc_quality, minmaj7: 0.146
